In [1]:
import yfinance as yf

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
import statsmodels.api as sm

In [5]:
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date

In [6]:
stock = 'fb'

cutoff=0.50
invest = 100
years_of_data_to_process = 25

In [7]:
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")
start_year =  int(year) - years_of_data_to_process
start = datetime.datetime(start_year, 1, 1)

In [8]:
data =  yf.download(stock, start=start)

[*********************100%***********************]  1 of 1 completed


In [9]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-05-18,42.049999,45.000000,38.000000,38.230000,38.230000,573576400
2012-05-21,36.529999,36.660000,33.000000,34.029999,34.029999,168192700
2012-05-22,32.610001,33.590000,30.940001,31.000000,31.000000,101786600
2012-05-23,31.370001,32.500000,31.360001,32.000000,32.000000,73600000
2012-05-24,32.950001,33.209999,31.770000,33.029999,33.029999,50237200
...,...,...,...,...,...,...
2021-06-02,330.380005,331.869995,327.119995,329.149994,329.149994,11654300
2021-06-03,325.779999,327.380005,323.480011,326.040009,326.040009,12610800
2021-06-04,325.899994,332.459991,325.179993,330.350006,330.350006,13282200


In [10]:
df = data["Close"].pct_change() * 100

In [11]:
df = df.rename("Today_Change_%")
df = df.reset_index()

In [12]:
df1 = pd.merge(data,df, on="Date")
#df1

In [13]:
df1.insert(7,'Volume_Lag', None)
df1['Volume_Lag'] = df1.Volume.shift(1).values/1000000000
df1.Volume = df1.Volume/1000000000

In [14]:
df1["Trend"] =  (df1["Close"] - df1["Low"])/ ((df1["High"] - df1["Low"]))

In [15]:
df1["Trend_Lag"] = df1["Trend"].shift(1)

In [16]:
df1['Close-Open'] =  df1['Close'] - df1['Open']
df1['Close-Open_Lag'] = df1['Close-Open'].shift(1)

In [17]:
df1['High-Low'] =  df1['High'] - df1['Low']
df1['High-Low_Lag'] = df1['High-Low'].shift(1)

In [18]:
df1 = sm.add_constant(df1)
df1["Up_Down"] = [1 if (i > 0) else 0 for i in df1["Today_Change_%"]]

In [19]:
df1.dropna(inplace= True)
#df1

In [20]:
X = df1[['const','Trend_Lag','Close-Open_Lag','High-Low_Lag','Volume_Lag']]

In [21]:
y = df1["Up_Down"].values

In [22]:
model = sm.Logit(y,X)

In [23]:
result =  model.fit()

Optimization terminated successfully.
         Current function value: 0.689167
         Iterations 4


In [24]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 2277
Model:                          Logit   Df Residuals:                     2272
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Jun 2021   Pseudo R-squ.:                0.003778
Time:                        10:22:34   Log-Likelihood:                -1569.2
converged:                       True   LL-Null:                       -1575.2
Covariance Type:            nonrobust   LLR p-value:                   0.01809
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.3873      0.125      3.097      0.002       0.142       0.632
Trend_Lag         -0.4341      0.186     -2.336      0.020      -0.798      -0.070
Close-Open_Lag    -0.0028      0.024     -0.119      0.905      -0.050       0.044
High-Low_Lag       0.0015      0.016      0.097      0.923      -0.029       0.033
Volume_Lag        -1.9664      1.521     -1.292      0.196      -4.949       1.016
==================================================================================
"""

In [25]:
prediction = result.predict(X)

In [26]:
df1['Prediction_Caculated'] = pd.array(prediction)
df1['Prediction_indicator'] = pd.array([1 if i > cutoff else 0 for i in prediction])
#df1.tail(10)

In [27]:
y = df1["Up_Down"].values

In [28]:
def confusion_matrix(act,pred):
    predtrans = ['Up' if i > cutoff else 'Down' for i in pred]
    actuals = ['Up' if i > 0 else 'Down' for i in act]
    confusion_matrix = pd.crosstab(pd.Series(actuals),
                                   pd.Series(predtrans),
                                   rownames = ["Actual"],
                                   colnames = ["Predict"]
                                  )
    return confusion_matrix

In [29]:
confusion_matrix(y,prediction)

Predict,Down,Up
Actual,,
Down,327,752
Up,323,875


In [30]:
z = confusion_matrix(y,prediction)
try:
    print((z.loc['Down','Down'] + z.loc['Up','Up']) / len(df1))
except:
    pass

0.5278875713658322


In [31]:
try:
    print( (z.loc['Down', 'Down']+ z.loc['Up','Up']) / (z.loc['Down', 'Down']+ z.loc['Up','Up'] + z.loc['Down','Up']) )
except:
    pass

0.6151484135107472


In [32]:
df1 = df1.assign(share=np.nan,money=np.nan)
#df1

In [33]:
diff_years = round((df1.iloc[-1,1] - df1.iloc[0,1])/np.timedelta64(1,'Y') + 0.5)   

In [34]:
#Simulate Investment transaction buy on opening when predict UP and sell daily average when predict DOWN

def buy_sell(open_price, sell_price,prediction, money, share):
    if prediction == 1 and money != 0:
        share =  money / open_price
        money = 0
    elif prediction == 0 and share != 0:
        money = share * sell_price
        share = 0
    else: pass
    return [money, share]
money = invest
share = 0
for i in range(len(df1)):
    [money, share] = buy_sell(df1.iloc[i,2],(df1.iloc[i,3]+df1.iloc[i,4])/2,df1.iloc[i,18], money, share)
    df1.iloc[i,19] = share
    df1.iloc[i,20] = money
    

In [35]:
print("\nIf ${:,.2f} was invested in [ {} ], and Just Hold and Not Trade for {:,} years, the ROI = ${:,.2f}".format( invest, stock.upper(), diff_years, invest/data.iloc[0, 0] * data.iloc[-1,0]))


If $100.00 was invested in [ FB ], and Just Hold and Not Trade for 10 years, the ROI = $800.71


In [36]:
print ("\nIf ${:,.2f} was invested {:2d} years ago, buy and sell according this script\'s recommandation, the ROI = ${:,.2f}".format(invest, diff_years, (money + (share * df1.iloc[-1,6]))))


If $100.00 was invested 10 years ago, buy and sell according this script's recommandation, the ROI = $887.52


In [37]:
df1_summary=df1[['Date', 'Up_Down','Prediction_indicator']].copy()
df1_summary['Stock Market Performance'] = df1_summary['Up_Down'].apply(lambda x: 'Up' if x > 0 else 'Down')
df1_summary['Scribe Predection'] = df1_summary['Prediction_indicator'].apply(lambda x: 'Up' if x > 0 else 'Down')
print (df1_summary[['Date','Stock Market Performance','Scribe Predection']].tail(15))

           Date Stock Market Performance Scribe Predection
2263 2021-05-18                     Down              Down
2264 2021-05-19                       Up                Up
2265 2021-05-20                       Up              Down
2266 2021-05-21                     Down              Down
2267 2021-05-24                       Up                Up
2268 2021-05-25                       Up              Down
2269 2021-05-26                     Down                Up
2270 2021-05-27                       Up                Up
2271 2021-05-28                     Down              Down
2272 2021-06-01                       Up                Up
2273 2021-06-02                       Up                Up
2274 2021-06-03                     Down                Up
2275 2021-06-04                       Up                Up
2276 2021-06-07                       Up                Up
2277 2021-06-08                     Down              Down


In [38]:
print ("\nToday [ %s ] actually went up," %stock.upper(), end = ' ') if (df1.iloc[-1,16] == 1) else print ("\nToday [ %s ] actually went down," %stock.upper(), end = " ")
print ("--- base on yesterday\'s data, ", end = '')
print ("We Predication [ %s ] should be going up." %stock.upper()) if (df1.iloc[-1,18] == 1) else print ("We Predicae [ %s ] should be going down." %stock.upper())
print ("\n=========> Actual and Predication MATCH <=========") if (df1.iloc[-1,16] == df1.iloc[-1,18]) else print("\n=========> Actual and Predication DO NOT match <=========")


Today [ FB ] actually went down, --- base on yesterday's data, We Predicae [ FB ] should be going down.

=========> Actual and Predication MATCH <=========


In [39]:
x_tran= df1[df1.Date.dt.year < 2021][['const','Trend_Lag','Close-Open_Lag','High-Low_Lag','Volume_Lag']]
y_train=df1[df1.Date.dt.year < 2021]["Up_Down"]
x_test= df1[df1.Date.dt.year >= 2021][['const','Trend_Lag','Close-Open_Lag','High-Low_Lag','Volume_Lag']]
y_test= df1[df1.Date.dt.year >= 2021]["Up_Down"]

In [40]:
model = sm.Logit(y_train,x_tran)
result=model.fit()

Optimization terminated successfully.
         Current function value: 0.689600
         Iterations 4


In [41]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Up_Down   No. Observations:                 2169
Model:                          Logit   Df Residuals:                     2164
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Jun 2021   Pseudo R-squ.:                0.003024
Time:                        10:22:35   Log-Likelihood:                -1495.7
converged:                       True   LL-Null:                       -1500.3
Covariance Type:            nonrobust   LLR p-value:                   0.05929
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.3578      0.128      2.806      0.005       0.108       0.608
Trend_Lag         -0.4071      0.191     -2.135      0.033      -0.781      -0.033
Close-Open_Lag     0.0048      0.026      0.183      0.855      -0.046       0.056
High-Low_Lag       0.0087      0.017      0.499      0.618      -0.025       0.043
Volume_Lag        -1.9883      1.530     -1.299      0.194      -4.988       1.011
==================================================================================
"""

In [42]:
prediction = result.predict(x_test)
confusion_matrix(y_test, prediction)

Predict,Down,Up
Actual,,
Down,1,52
Up,0,55


In [43]:
z = confusion_matrix(y_test,prediction)

In [44]:
try:
    print ("\n=========> Prediction Accuracy Rate: %.4f <=========\n"  %((z.loc['Down','Down'] + z.loc['Up','Up']) / len(x_test)))
except:
    print ("\n=========> Predication effectiveness is not avairable <=========\n" )


=========> Prediction Accuracy Rate: 0.5185 <=========



In [45]:
prediction = result.predict(x_test)
now_up_down  = result.predict([1.0, df1.iloc[-1, 10], df1.iloc[-1, 12], df1.iloc[-1, 14], df1.iloc[-1, 7]])
print ("\n=========> Current trend = %.4f,  " %now_up_down, end=' ')
print ("[ %s ] will go up! <=========" %stock.upper()) if now_up_down > cutoff else print ("[ %s ] will go down! <=========" %stock.upper()) 


=========> Current trend = 0.5742,   [ FB ] will go up! <=========


In [46]:
#df1.to_csv('fb.csv', index = False)